In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#import the data
data = pd.read_csv("ukraine-damages.csv", delimiter = "|")
data.head()

,damage_id,iso3,country,gid_1,oblast,rayon,type_of_infrastructure,if_other_what,date_of_event,source_name,source_date,source_link,additional_sources,extent_of_damage,_internal_filter_date,_weights,access_subindicator,pcode
0,D0011,UKR,Ukraine,['UKR.15_1'],Luhanska,Siverskodonetskyi,Warehouse,NaN,2022-03-25,OCHA,2022-03-28,https://reliefweb.int/report/ukraine/ukraine-h...,NaN,Destroyed,2022-03-25,0.7,['7.2'],UA44
1,D0012,UKR,Ukraine,['UKR.15_1'],Luhanska,Siverskodonetskyi,Warehouse,NaN,2022-03-26,OCHA,2022-03-28,https://reliefweb.int/report/ukraine/ukraine-h...,NaN,Partially damaged,2022-03-26,0.7,['7.2'],UA44
2,D0015,UKR,Ukraine,['UKR.14_1'],Lvivska,NaN,Warehouse,NaN,2022-03-26,OCHA,2022-03-28,https://reliefweb.int/report/ukraine/ukraine-h...,NaN,Unknown,2022-03-26,1.0,['7.2'],UA46
3,D0016,UKR,Ukraine,['UKR.14_1'],Lvivska,NaN,Aircraft repair plant,Aircraft repair plan,2022-03-26,OCHA,2022-03-28,https://reliefweb.int/report/ukraine/ukraine-h...,NaN,Destroyed,2022-03-26,1.0,['7.2'],UA46
4,D0017,UKR,Ukraine,['UKR.12_1'],Kyivska,NaN,Bridge,NaN,2022-03-22,OCHA,2022-03-28,https://reliefweb.int/report/ukraine/ukraine-h...,NaN,Destroyed,2022-03-22,1.0,['9.2'],UA32


In [11]:
#Oblast is the region
#Rayon is the district within the region
#I think it makes the most sense to model at the Rayon level as these are smaller than Oblasts
#how many are NaN?
a = data['rayon'].isna().sum()
b = data['rayon'].value_counts()
a,b


(np.int64(8720),
 rayon
 Nikopolskyi     2474
 Kramatorskyi    2274
 Kharkivskyi     1308
 Khersonskyi     1187
 Pokrovskyi       733
                 ... 
 Khmilnytskyi       1
 Yaltynskyi         1
 Zhmerynskyi        1
 Sambirskyi         1
 Kovelskyi          1
 Name: count, Length: 113, dtype: int64)

In [ ]:
#getting unique rayons
#rayons = data["Rayon"].unique()
#print(f"There are {len(rayons)} unique rayons in the dataset.")
data['rayon'][0]